# SageMaker Model Deployment

This notebook demonstrates how to deploy a trained XGBoost model using AWS SageMaker.

## Task 1: Setup Environment

In [ ]:
# Install dependencies
%pip install seaborn
%reset -f

import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
boto3_session = boto3.Session()
sagemaker_client = boto3_session.client('sagemaker')
sagemaker_runtime = boto3_session.client('sagemaker-runtime')
s3_client = boto3.client('s3')